# The Copper-to-Gold Ratio Using the OpenBB SDK

The copper-to-gold ratio is known as a leading economic indicator.  It is most commonly paired as a time series with the ten-year US Treasury yield. The notable events are the divergences in directional movement, signaling a fundamental regime change that will unfold over months and years.  Not something to go YOLO into, but a metric to shape a long-term view of global economic conditions.

The ratio is defined as dividing the spot price of one ounce of copper by an ounce gold.  How much copper is bought with one ouce of gold.  Sounds simple enough,  divide the price of copper by the price of gold, done.  The OpenBB Platform can make quick work out of this task, really quick.  Let's explore.

In [71]:
# Import the Platform and Pandas for some calculations.

import pandas as pd
import plotly.graph_objects as go
from openbb import obb

The most accessible data is going to be the continuous front-month futures contracts for physical delivery, listed on the CME.  We'll create a Pandas Series for each asset, requesting weekly historical data using the `openbb-yfinance` data extension.

In [62]:
data = pd.DataFrame()
cols_dict = {"GC": "Gold", "HG": "Copper"}
for symbol in ["GC", "HG"]:
    data[symbol] = (
        obb.derivatives.futures.historical(
            symbol,
            start_date="2000-01-01",
            interval="1wk",
        ).to_df()["close"].rename(symbol)
    )
data.columns = cols_dict.values()
data.index = data.index.strftime("%Y-%m-%d")

Let's inspect the results.

In [63]:
data.head(2)

,Gold,Copper
date,,
2000-08-28,277.000000,0.889
2000-09-04,273.299988,0.912


To get the copper-to-gold ratio, divide the two columns along each row.

In [64]:
data["Copper/Gold Ratio"] = data["Copper"]/data["Gold"]

data.tail(2)

,Gold,Copper,Copper/Gold Ratio
date,,,
2023-11-20,2002.199951,3.7965,0.001896
2023-11-27,2011.800049,3.7625,0.001870


Because the numbers are so small, the ratio is often be presented as a % value.  0.2% is a popular way to display the value.  However, to plot it on the same y-axis as a Treasury yield, it needs to be multiplied by 1000.  Let's alter the block above to include this.

In [65]:
data["Copper/Gold Ratio"] = (data["Copper"]/data["Gold"]) * 1000

data.tail(2)

,Gold,Copper,Copper/Gold Ratio
date,,,
2023-11-20,2002.199951,3.7965,1.896164
2023-11-27,2011.800049,3.7625,1.870216


Now let's add a column for the daily 10 Year US Treasury Yield.  This can be requested using the `fred_series` function within the `economy` module.  The first line in the block below requests the data, the second assigns it to a column in the target DataFrame.

In [67]:
us10year = obb.economy.fred_series(
    "DGS10",
    frequency="wem",
    start_date="2000-08-28",
    end_date="2023-11-27"
).to_df()[["DGS10"]]

us10year.index = us10year.index.strftime("%Y-%m-%d")
data["US 10-Year Constant Maturity"] = us10year["DGS10"]

data.head(2)

,Gold,Copper,Copper/Gold Ratio,US 10-Year Constant Maturity
date,,,,
2000-08-28,277.000000,0.889,3.209386,5.78
2000-09-04,273.299988,0.912,3.336992,5.68


With all the data collected, let's draw the chart to visualize the relationship.

In [72]:

fig = go.Figure()
fig.add_scatter(x=data.index, y=data["Copper/Gold Ratio"], name="Copper/Gold Ratio (x1000) %")
fig.add_scatter(x=data.index, y=data["US 10-Year Constant Maturity"], name="US 10-Year Constant Maturity %")
fig.update({"layout":
    {"xaxis": {"title": "Date"},
    "yaxis": {"title": "%"},
    "title": "Copper/Gold Ratio vs. US 10-Year Constant Maturity",
    "title_y":0.90,
    "title_x":0.5,
}})
fig.update_layout(
    legend=dict(
        yanchor="top",
        y=1,
        xanchor="right",
        x=1.0
    )
)

What we have currently is the price relationship between one Troy ounce of gold and one pound of copper.  As we described the copper-to-gold ratio as the price-per-ounce of each, some adjustments are required to be true to the definition.

- 1 ounce = 0.911458 Troy ounces
- 1 pound = 16 ounces
  
To adjust the gold price as USD/ounce, multiply each row by 0.911458.  To adjust the copper price, divide each row by 16.

In [78]:
data["Copper/Gold Ratio per Ounce (x1000) %"] = (
    ((data["Copper"]/16)/(data["Gold"]*0.911458))*1000
)

data.tail(2)

,Gold,Copper,Copper/Gold Ratio,US 10-Year Constant Maturity,Copper/Gold Ratio per Ounce (x1000) %
date,,,,,
2023-11-20,2002.199951,3.7965,1.896164,4.42,0.130023
2023-11-27,2011.800049,3.7625,1.870216,NaN,0.128243


Each y-axis requires its own DataFrame, so let's make one for each next.

In [23]:
y1 = pd.DataFrame(data["Copper/Gold Ratio per Ounce (x1000) %"])
y2 = pd.DataFrame(data["US 10-Year Constant Maturity"])
fig = show_plot(y1, y2, external_axes = True)

In [85]:
fig = go.Figure()

# Add the first scatter trace with its own y-axis
fig.add_scatter(x=data.index, y=data["Copper/Gold Ratio"], name="Copper/Gold Ratio (x1000) %", yaxis="y1")

# Add the second scatter trace with its own y-axis
fig.add_scatter(x=data.index, y=data["US 10-Year Constant Maturity"], name="US 10-Year Constant Maturity %", yaxis="y2")

# Update the layout to include the y-axes and their titles
fig.update_layout(
    yaxis=dict(title="Copper/Gold Ratio (x1000) %", side="left", position=0, titlefont=dict(size=12), showgrid=False),
    yaxis2=dict(title="US 10-Year Constant Maturity %", side="right", overlaying="y", position=1, titlefont=dict(size=12)),
    xaxis=dict(title="Date"),
    title="Copper/Gold Ratio vs. US 10-Year Constant Maturity",
    title_y=0.90,
    title_x=0.5,
)

# Set the legend position
fig.update_layout(
    legend=dict(yanchor="top", y=1, xanchor="right", x=1.0, font=dict(size=10))
)

# Show the plot
fig.show()

There you have it, folks!  The OpenBB Platform provides endless possibilities for creating unique indicators and analysis with the wide variety of data available at your fingertips.  We love seeing the creations of users, so be sure to tag us on social media and show off your work.